In [21]:
#导入相关库类
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.config(conf=SparkConf()).getOrCreate()

In [22]:
#定义mysql数据库连接方法(参数为库名，表名)，并创建dataframe临时表
def connect_DB(db_name,tb_name):
    #连接数据库
    jdbcDF = spark.read.format("jdbc")\
    .option("driver","com.mysql.jdbc.Driver")\
    .option("url","jdbc:mysql://139.155.70.177:3306/%s"%(db_name))\
    .option("dbtable","%s"%(tb_name))\
    .option("user","root")\
    .option("password","zzjz123")\
    .load()
    #创建临时表
    jdbcDF.createOrReplaceTempView(tb_name)

In [23]:
connect_DB("Gadaite","Orders")

In [24]:
#查看Orders表信息
spark.sql("""
    select * from Orders
    """).show()

+---------+-------------------+----------+
|order_num|         order_date|   cust_id|
+---------+-------------------+----------+
|    20005|2020-05-01 00:00:00|1000000001|
|    20006|2020-01-12 00:00:00|1000000003|
|    20007|2020-01-30 00:00:00|1000000004|
|    20008|2020-02-03 00:00:00|1000000005|
|    20009|2020-02-08 00:00:00|1000000001|
+---------+-------------------+----------+



In [25]:
connect_DB("Gadaite","OrderItems")

In [26]:
#查看OrderItems表信息
spark.sql("""
    select * from OrderItems
    """).show()

+---------+----------+-------+--------+----------+
|order_num|order_item|prod_id|quantity|item_price|
+---------+----------+-------+--------+----------+
|    20005|         1|   BR01|     100|      5.49|
|    20005|         2|   BR03|     100|     10.99|
|    20006|         1|   BR01|      20|      5.99|
|    20006|         2|   BR02|      10|      8.99|
|    20006|         3|   BR03|      10|     11.99|
|    20007|         1|   BR03|      50|     11.49|
|    20007|         2| BNBG01|     100|      2.99|
|    20007|         3| BNBG02|     100|      2.99|
|    20007|         4| BNBG03|     100|      2.99|
|    20007|         5| RGAN01|      50|      4.49|
|    20008|         1| RGAN01|       5|      4.99|
|    20008|         2|   BR03|       5|     11.99|
|    20008|         3| BNBG01|      10|      3.49|
|    20008|         4| BNBG02|      10|      3.49|
|    20008|         5| BNBG03|      10|      3.49|
|    20009|         1| BNBG01|     250|      2.49|
|    20009|         2| BNBG02| 

In [27]:
connect_DB("Gadaite","Customers")
spark.sql("""
    select * from Customers
    """).show()

+----------+-------------+--------------------+---------+----------+--------+------------+------------------+--------------------+
|   cust_id|    cust_name|        cust_address|cust_city|cust_state|cust_zip|cust_country|      cust_contact|          cust_email|
+----------+-------------+--------------------+---------+----------+--------+------------+------------------+--------------------+
|1000000001| Village Toys|      200 Maple Lane|  Detroit|        MI|   44444|         USA|        John Smith|sales@villagetoys...|
|1000000002|   Kids Place|333 South Lake Drive| Columbus|        OH|   43333|         USA|    Michelle Green|                null|
|1000000003|      Fun4All|       1 Sunny Place|   Muncie|        IN|   42222|         USA|         Jim Jones|  jjones@fun4all.com|
|1000000004|      Fun4All| 829 Riverside Drive|  Phoenix|        AZ|   88888|         USA|Denise L. Stephens|dstephens@fun4all...|
|1000000005|The Toy Store|    4545 53rd Street|  Chicago|        IL|   54545|      

In [28]:
"""
    Orders:订单编号，客户ID，日期
    OrderItems:订单物品存储
    Customers:顾客实际信息
    Q：列出订购物品RGAN01的所有顾客ID
"""

'\n    Orders:订单编号，客户ID，日期\n    OrderItems:订单物品存储\n    Customers:顾客实际信息\n    Q：列出订购物品RGAN01的所有顾客ID\n'

In [29]:
#非子查询
spark.sql("""
    select order_num 
    from OrderItems 
    where prod_id="RGAN01"
    """).show()
"结果：20007，20008"
spark.sql("""
    select cust_id 
    from Orders 
    where order_num in (20007,20008)
    """).show()
"结果：1000000004，1000000004"

+---------+
|order_num|
+---------+
|    20007|
|    20008|
+---------+

+----------+
|   cust_id|
+----------+
|1000000004|
|1000000005|
+----------+



'结果：1000000004，1000000004'

In [30]:
#使用子查询,从内向外处理
spark.sql("""
    select cust_id 
    from Orders 
    where order_num in (
        select order_num 
        from OrderItems 
        where prod_id='RGAN01'
    )
    """).show()

+----------+
|   cust_id|
+----------+
|1000000004|
|1000000005|
+----------+



In [31]:
#可在基础上再检索顾客的所有信息，或者部分信息
#以子查询为例,过多嵌套会降低性能
spark.sql("""
    select * 
    from Customers 
    where cust_id in (
        select cust_id 
        from Orders 
        where order_num in (
            select order_num 
            from OrderItems 
            where prod_id='RGAN01'
        )
    )
    """).show()

+----------+-------------+-------------------+---------+----------+--------+------------+------------------+--------------------+
|   cust_id|    cust_name|       cust_address|cust_city|cust_state|cust_zip|cust_country|      cust_contact|          cust_email|
+----------+-------------+-------------------+---------+----------+--------+------------+------------------+--------------------+
|1000000005|The Toy Store|   4545 53rd Street|  Chicago|        IL|   54545|         USA|        Kim Howard|                null|
|1000000004|      Fun4All|829 Riverside Drive|  Phoenix|        AZ|   88888|         USA|Denise L. Stephens|dstephens@fun4all...|
+----------+-------------+-------------------+---------+----------+--------+------------+------------------+--------------------+



In [32]:
#作为计算字段使用子查询
#显示Customers表中每个顾客的订单总数

In [33]:
#以下执行可能会报错,且是一种错误的写法
#如果报错按下面格式书写，使用完全限定列名，其原因关于DBMS无法判断：
spark.sql("""
    select cust_name,cust_state,
    (select count(*)
        from Orders
    ) as count 
    from Customers 
    order by cust_name
    """).show()

+-------------+----------+-----+
|    cust_name|cust_state|count|
+-------------+----------+-----+
|      Fun4All|        IN|    5|
|      Fun4All|        AZ|    5|
|   Kids Place|        OH|    5|
|The Toy Store|        IL|    5|
| Village Toys|        MI|    5|
+-------------+----------+-----+



In [34]:
#正确的写法
spark.sql("""
    select cust_name,cust_state,
    (select count(*)
        from Orders 
        where Customers.cust_id=Orders.cust_id
    ) as count 
    from Customers 
    order by cust_name
    """).show()
#理解：
"""
    该查询会新增新的一个字段，必定是作为计算字段的查询
    前面例子是作为筛选条件查询，不会生成一个新的字段
    对应位置即可
    思路：
    在外层按照行来看，每行会取到一个对应的cust_id,
    按照这个cust_id对另一个表先进行条件筛选，再count的行数就自然是个数了
"""

+-------------+----------+-----+
|    cust_name|cust_state|count|
+-------------+----------+-----+
|      Fun4All|        IN|    1|
|      Fun4All|        AZ|    1|
|   Kids Place|        OH|    0|
|The Toy Store|        IL|    1|
| Village Toys|        MI|    2|
+-------------+----------+-----+



'\n    该查询会新增新的一个字段，必定是作为计算字段的查询\n    前面例子是作为筛选条件查询，不会生成一个新的字段\n    对应位置即可\n    思路：\n    在外层按照行来看，每行会取到一个对应的cust_id,\n    按照这个cust_id对另一个表先进行条件筛选，再count的行数就自然是个数了\n'

In [35]:
#Q：返回价格>=10的产品(不是总价)的顾客列表，先使用OrderItems
spark.sql("""
    select cust_id 
    from Orders  
    where order_num in (
        select order_num 
        from OrderItems 
        where item_price >=10
    )
    """).show()

+----------+
|   cust_id|
+----------+
|1000000003|
|1000000004|
|1000000001|
|1000000005|
+----------+



In [36]:
#Q：确定哪些订单购买了prod_id=BR01产品
#   返回每个产品的顾客id和订单日期，按订单日期排序
spark.sql("""
    select cust_id,order_date 
    from Orders 
    where order_num in (
        select order_num 
        from OrderItems 
        where prod_id='BR01'
    ) 
    order by order_date
    """).show()

+----------+-------------------+
|   cust_id|         order_date|
+----------+-------------------+
|1000000003|2020-01-12 00:00:00|
|1000000001|2020-05-01 00:00:00|
+----------+-------------------+



In [37]:
#Q：在上一个Q上，返回prod_id为BR01的产品的所有顾客的电子邮件
spark.sql("""
    select cust_email 
    from Customers 
    where cust_id in (
        select cust_id 
        from Orders 
        where order_num in (
            select order_num 
            from OrderItems 
            where prod_id='BR01' 
        ----order by order_date
        ) 
            order by order_date 
    ) 
    """).show()

+--------------------+
|          cust_email|
+--------------------+
|sales@villagetoys...|
|  jjones@fun4all.com|
+--------------------+



In [38]:
#需要一个顾客列表，包含已订购的总金额，返回顾客ID和总数(从大到小)
spark.sql("""
    select cust_id,
    (select sum(quantity*item_price) 
        from OrderItems 
        where OrderItems.order_num = Orders.order_num
    )as total_ordered 
    from Orders 
    order by total_ordered desc
    """).show()

+----------+-------------+
|   cust_id|total_ordered|
+----------+-------------+
|1000000001|      1867.50|
|1000000004|      1696.00|
|1000000001|      1648.00|
|1000000003|       329.60|
|1000000005|       189.60|
+----------+-------------+



In [39]:
#注册临时表Products
connect_DB("Gadaite","Products")

In [40]:
#从Products表中检索prod_name，计算列quant_sold
#包含所售产品总数
#在Customers上使用sum(quantity)检索
spark.sql("""
    select prod_name,
    (
        select 
        sum(quantity) 
        from OrderItems 
        where OrderItems.prod_id=Products.prod_id 
    ) as quant_sold 
    from Products
    """).show()

+-------------------+----------+
|          prod_name|quant_sold|
+-------------------+----------+
|  Bird bean bag toy|       360|
|Rabbit bean bag toy|       360|
|  8 inch teddy bear|       120|
|         Queen doll|      null|
|        Raggedy Ann|        55|
|          King doll|      null|
| 18 inch teddy bear|       165|
| 12 inch teddy bear|        10|
|  Fish bean bag toy|       360|
+-------------------+----------+



In [41]:
spark.stop()